In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib.dates as mdates

import random
import torch
import numpy as np
import pandas as pd
import multiprocessing
from itertools import islice

In [ ]:
from gluonts.evaluation import make_evaluation_predictions, Evaluator
from gluonts.dataset.repository.datasets import get_dataset
from pytorch_lightning.loggers import WandbLogger, CSVLogger

from estimator import LagGPTEstimator

In [ ]:
class CombinedDatasetIterator:
    def __init__(self, datasets, seed, weights):
        self._datasets = [iter(el) for el in datasets]
        self._weights = weights
        self._rng = random.Random(seed)

    def __next__(self):
        (dataset,) = self._rng.choices(self._datasets, weights=self._weights, k=1)
        return next(dataset)

In [ ]:
class CombinedDataset:
    def __init__(self, datasets, seed=None, weights=None):
        self._seed = seed
        self._datasets = datasets
        self._weights = weights
        n_datasets = len(datasets)
        if weights is None:
            self._weights = [1 / n_datasets] * n_datasets

    def __iter__(self):
        return CombinedDatasetIterator(self._datasets, self._seed, self._weights)
    
    def __len__(self):
        return sum([len(ds) for ds in self._datasets])

In [ ]:
gluonts_ds = [
        get_dataset("airpassengers").train,
        get_dataset("australian_electricity_demand").train,
        get_dataset("car_parts_without_missing").train,
        get_dataset("cif_2016").train,
        get_dataset("covid_deaths").train,
        get_dataset("electricity").train,
        get_dataset("electricity_weekly").train,
        get_dataset("exchange_rate").train,
        get_dataset("fred_md").train,
        get_dataset("hospital").train,
        get_dataset("kaggle_web_traffic_weekly").train,
        get_dataset("kdd_cup_2018_without_missing").train,
        get_dataset("london_smart_meters_without_missing").train,
        get_dataset("nn5_daily_with_missing").train,
        get_dataset("nn5_weekly").train,
        get_dataset("pedestrian_counts").train,
        get_dataset("rideshare_without_missing").train,
        get_dataset("saugeenday").train,
        get_dataset("solar-energy").train,
        get_dataset("solar_10_minutes").train,
        get_dataset("solar_weekly").train,
        get_dataset("taxi_30min").train,
        get_dataset("temperature_rain_without_missing").train,
        get_dataset("tourism_monthly").train,
        get_dataset("uber_tlc_daily").train,
        get_dataset("uber_tlc_hourly").train,
        get_dataset("vehicle_trips_without_missing").train,
        get_dataset("weather").train,
        get_dataset("wiki-rolling_nips").train,
        get_dataset("m4_daily").train,
        get_dataset("m4_hourly").train,
        get_dataset("m4_monthly").train,
        get_dataset("m4_quarterly").train,
        get_dataset("m4_yearly").train,
        get_dataset("wind_farms_without_missing").train,
]
dataset = CombinedDataset(gluonts_ds, weights=[sum([len(x["target"]) for x in d]) for d in gluonts_ds])

In [ ]:
val_dataset = get_dataset("m4_weekly").test

In [ ]:
meta = get_dataset("m4_weekly").metadata

In [ ]:
meta

In [ ]:
#set more seeds

In [ ]:
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
logger = CSVLogger("logs", name="Lag-gpt-large-"+str(seed))

In [ ]:
!unset LD_LIBRARY_PATH

In [ ]:
estimator = LagGPTEstimator(
    prediction_length=meta.prediction_length,
    context_length=1024, # block_size: int = 1024, 2048 
    batch_size=4, # 4, 8, 16, 32
    n_layer=8,
    n_head=4,
    n_embd=32, # 32,64,128,256,512,1024,2048,4096
    scaling="std",
    
    # set aug prob
    aug_prob=1.0,
    aug_rate=0.1,
    
    num_batches_per_epoch=100,
    trainer_kwargs=dict(max_epochs=300, accelerator="gpu", precision="32", logger=logger),
)

In [ ]:
predictor = estimator.train(
    training_data=dataset, 
    validation_data=val_dataset,
    shuffle_buffer_length=1000
)

In [ ]:
test_dataset = get_dataset("traffic").test

In [ ]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=test_dataset, predictor=predictor
)

In [ ]:
forecasts = list(forecast_it)

In [ ]:
tss = list(ts_it)

In [ ]:
# num_workers is limited to 10 if cpu has more cores
num_workers = min(multiprocessing.cpu_count(), 10)

evaluator = Evaluator(num_workers=num_workers)
agg_metrics, ts_metrics = evaluator(
    iter(tss), iter(forecasts), num_series=len(test_dataset)
)

In [ ]:
agg_metrics

In [ ]:
metric_type = []
error = []
for state in agg_metrics:
    metric_type.append(state)
for value in agg_metrics.values():
    error.append(value)

In [ ]:
df = pd.DataFrame( error, metric_type).reset_index().rename(columns = {'index': 'error',0:'metric_type'}).T
df.columns = df.iloc[0,:]
df = df.iloc[1:, :]
df['#parameters']=411000000#102000000#411000000#6800000
df['seed'] = seed
# df.to_csv('error100.csv')

In [ ]:
dff = pd.read_csv('error100.csv')#.drop('Unnamed: 0', axis=1)
# dff.columns
dff = dff.append(df)
dff.to_csv('error100.csv', index=False)

In [ ]:
ts_metrics.plot(x="MSIS", y="MAPE", kind="scatter")
plt.grid(which="both")
plt.show()

In [ ]:
plt.figure(figsize=(20, 15))
plt.rcParams.update({'font.size': 15})

for idx, (forecast, ts) in islice(enumerate(zip(forecasts, tss)), 9):
    ax = plt.subplot(3, 3, idx+1)
    forecast.plot(color='g')
    ts[-3 * 24:][0].plot(label="target")
    plt.xticks(rotation=60)
    ax.set_title(forecast.item_id)

plt.gcf().tight_layout()
plt.legend()
plt.show()

In [ ]:
df = pd.read_csv('error100.csv')
df = df.drop_duplicates(subset=['#parameters','seed'], keep='last')
temp = df.groupby(['#parameters'])['RMSE'].mean().reset_index().dropna()
std_temp = df.groupby(['#parameters'])['RMSE'].std().reset_index()

In [ ]:
temp

In [ ]:
std_temp

In [ ]:
plt.clf()
fig = plt.subplots(figsize =(5, 3))# large

# plt.bar(temp['model'],temp['crps'],width = 0.2)
plt.plot(temp['#parameters'],temp['RMSE'],'bo-')
for x,y in zip(np.log10(temp['#parameters']),[0,0.1,0.5]):

    label = f"p={y}"

    plt.annotate(label, # this is the text
                 (x,y), # these are the coordinates to position the label
                 textcoords="offset points", # how to position the text
                 xytext=(0,10), # distance from text to points (x,y)
                 ha='center') 
plt.title('Test RMSE vs params')
plt.xlabel('number of parameters')
plt.ylabel('RMSE on test set')
plt.show()

In [ ]:
import seaborn as sns
fig, ax = plt.subplots(figsize=(5,3))
clrs = sns.color_palette("husl", 5)
with sns.axes_style("darkgrid"):
    ax.plot(temp['#parameters'], temp['RMSE'])
    ax.fill_between(temp['#parameters'], temp['RMSE']-std_temp['RMSE'], temp['RMSE']+std_temp['RMSE'] ,alpha=0.3)
    ax.legend()
plt.title('RMSE vs params')
plt.xlabel('number of parameters')
plt.ylabel('RMSE on test set')
plt.show()
    # ax.set_yscale('log')

In [ ]:
import os
path = "logs/"
dir_list = os.listdir(path)
dir_list

In [ ]:
dff = pd.DataFrame()
for i in dir_list:
    
    df = pd.read_csv('logs/'+i+'/version_0/metrics.csv').drop(['train_loss'], axis=1).dropna()
    df['seed']=int(i.split("-")[-1])
    if i.split("-")[-2]=='medium':
        df['parameters']=102000000
    if i.split("-")[-2]=='small':
        print(i)
        df['parameters']=6800000
    if i.split("-")[-2]=='large':
        df['parameters']=411000000
    dff = dff.append(df, ignore_index=True)


In [ ]:
dff['parameters'].unique().tolist()

In [ ]:
temp = dff.groupby(['parameters','epoch'])['val_loss'].mean().reset_index().dropna()
std_temp = dff.groupby(['parameters','epoch'])['val_loss'].std().reset_index().dropna()


In [ ]:
temp

In [ ]:
fig, ax = plt.subplots(figsize=(6,4))
clrs = sns.color_palette("husl", 5)
with sns.axes_style("darkgrid"):
    for i in dff['parameters'].unique().tolist():
        ax.plot(temp[temp['parameters']==i]['epoch'], temp[temp['parameters']==i]['val_loss'], label=i)
        ax.fill_between(temp[temp['parameters']==i]['epoch'], temp[temp['parameters']==i]['val_loss']-std_temp[std_temp['parameters']==i]['val_loss'], temp[temp['parameters']==i]['val_loss']+std_temp[std_temp['parameters']==i]['val_loss'] ,alpha=0.3)
        ax.legend()
plt.title('validation loss vs epochs')
plt.xlabel('epochs')
plt.ylabel('nll')
plt.show()